In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

2024-09-23 18:36:04.117384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 18:36:04.117500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 18:36:04.275225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df1 = pd.read_csv('/kaggle/working/personnel_financial_info.csv')
df2 = pd.read_csv('/kaggle/working/personnel_behavior_info.csv')
df3 = pd.read_csv('/kaggle/working/personnel_offences_info.csv')
df4 = pd.read_csv('/kaggle/working/personnel_general_info.csv')

df_combined = pd.merge(df1, df2, on='Enrollment_No')
df_combined = pd.merge(df_combined, df3, on='Enrollment_No')
df_combined = pd.merge(df_combined, df4, on='Enrollment_No')
df_combined.head()

,Enrollment_No,Income,Debt,Prev_1_Year_Expenditure,Prev_2_Year_Expenditure,Calls_From_Unverified,Avg_Hours_Outside,Substance_Abuse,Gambling,Insubordination_Offences,Civil_Offences,PSS_Score,Security_Allowance,Experience
0,234561,61926,774706,865062,1688542,2,72,0,0,"{'minor': 4, 'major': 0}",2,24.555315,0.596044,23
1,234562,51324,957315,228299,735520,4,4,0,0,"{'minor': 4, 'major': 1}",2,27.135999,0.268088,22
2,234563,51407,945820,312342,1563222,2,91,1,1,"{'minor': 4, 'major': 0}",2,23.435270,0.332606,28
3,234564,85643,754354,951869,704616,2,94,0,0,"{'minor': 0, 'major': 2}",2,24.394902,0.819141,9
4,234565,51853,780184,444269,812231,3,31,0,1,"{'minor': 1, 'major': 2}",0,24.301075,0.658854,14


In [6]:
df_new = pd.DataFrame()
df_new['Enrollment_No'] = df_combined['Enrollment_No']
df_new['Debt_to_Income_Ratio'] = df_combined['Debt'] / df_combined['Income']
df_new['Prev_Spending_Habits'] = df_combined['Prev_1_Year_Expenditure'] / df_combined['Prev_2_Year_Expenditure']
df_new['Calls_From_Unverified'] = df_combined['Calls_From_Unverified']
df_new['Avg_Hours_Outside'] = df_combined['Avg_Hours_Outside']
df_new['Insubordination'] = df_combined['Insubordination_Offences'].apply(lambda x: int(x[10]) * 1 + int(x[-2]) * 5)
df_new['Civil_Offences'] = df_combined['Civil_Offences']
df_new['PSS_Score'] = df_combined['PSS_Score']
df_new['Experience'] = df_combined['Experience']
df_new['Security_Allowance'] = df_combined['Security_Allowance']
df_new['Gambling'] = df_combined['Gambling']
df_new['Substance_Abuse'] = df_combined['Substance_Abuse']
df_new.head()

,Enrollment_No,Debt_to_Income_Ratio,Prev_Spending_Habits,Calls_From_Unverified,Avg_Hours_Outside,Insubordination,Civil_Offences,PSS_Score,Experience,Security_Allowance,Gambling,Substance_Abuse
0,234561,12.510190,0.512313,2,72,4,2,24.555315,23,0.596044,0,0
1,234562,18.652385,0.310391,4,4,9,2,27.135999,22,0.268088,0,0
2,234563,18.398662,0.199807,2,91,4,2,23.435270,28,0.332606,1,1
3,234564,8.808122,1.350905,2,94,10,2,24.394902,9,0.819141,0,0
4,234565,15.046073,0.546974,3,31,11,0,24.301075,14,0.658854,1,0


In [7]:
roll_numbers = np.array(df_new['Enrollment_No'])
data = np.array(df_new.drop(columns = 'Enrollment_No'))

In [8]:
data.shape

(2000, 11)

In [9]:
scaler = MinMaxScaler()
data[:, :9] = scaler.fit_transform(data[:, :9])

In [10]:
train_size = int(len(data) * 0.5)

X_train = data[:train_size]
X_test = data[train_size:]

roll_train = roll_numbers[:train_size]
roll_test = roll_numbers[train_size:]


In [11]:
input_dim = X_train.shape[1]
encoding_dim = 5

In [12]:
input_layer = Input(shape=(input_dim,))
encoded = Dense(8, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [13]:
history = autoencoder.fit(X_train, X_train, 
                          epochs=100, 
                          batch_size=32, 
                          shuffle=True, 
                          validation_split=0.2)

Epoch 1/100
25/25 [==============================] - 1s 11ms/step - loss: 0.1170 - val_loss: 0.1134
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 0.1129 - val_loss: 0.1098
Epoch 3/100
25/25 [==============================] - 0s 3ms/step - loss: 0.1091 - val_loss: 0.1056
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.1044 - val_loss: 0.1007
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0991 - val_loss: 0.0946
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0905 - val_loss: 0.0838
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0791 - val_loss: 0.0732
Epoch 8/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0707 - val_loss: 0.0670
Epoch 9/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0663 - val_loss: 0.0639
Epoch 10/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0640 - val_loss: 0.0619
Epoch 11

In [14]:
reconstructed_data = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.square(X_test - reconstructed_data), axis=1)

sorted_indices = np.argsort(reconstruction_error)[::-1]  

n_top = int(0.05 * len(reconstruction_error))  
top_soldiers_indices = sorted_indices[:n_top]

top_soldiers_rolls = roll_test[top_soldiers_indices]
top_soldiers_errors = reconstruction_error[top_soldiers_indices]

top_soldiers_data = {
    "Soldier_Roll_Number": top_soldiers_rolls,
    "Reconstruction_Error": top_soldiers_errors
}

df_top_soldiers = pd.DataFrame(top_soldiers_data)

df_top_soldiers.to_csv("top_likely_defectors_roll_numbers.csv", index=False)

print(f"Top {n_top} soldiers likely to defect (sorted by reconstruction error):")
for i, (roll, error) in enumerate(zip(top_soldiers_rolls, top_soldiers_errors), start=1):
    print(f"Rank {i}: Soldier Roll Number {roll}, Reconstruction Error: {error}")

32/32 [==============================] - 0s 1ms/step
Top 50 soldiers likely to defect (sorted by reconstruction error):
Rank 1: Soldier Roll Number 311126, Reconstruction Error: 0.1723290741884778
Rank 2: Soldier Roll Number 311136, Reconstruction Error: 0.15642218156244192
Rank 3: Soldier Roll Number 311133, Reconstruction Error: 0.15020500186758481
Rank 4: Soldier Roll Number 311138, Reconstruction Error: 0.11650075359674221
Rank 5: Soldier Roll Number 235861, Reconstruction Error: 0.1153715986600281
Rank 6: Soldier Roll Number 311139, Reconstruction Error: 0.11079907387252169
Rank 7: Soldier Roll Number 311112, Reconstruction Error: 0.10920132033177793
Rank 8: Soldier Roll Number 311122, Reconstruction Error: 0.09823506123100531
Rank 9: Soldier Roll Number 311132, Reconstruction Error: 0.09366384676179323
Rank 10: Soldier Roll Number 236312, Reconstruction Error: 0.09033117450480177
Rank 11: Soldier Roll Number 311115, Reconstruction Error: 0.08792883040105301
Rank 12: Soldier Roll 